In [ ]:
#import all libraries
import pandas as pd
import requests
import bs4
import re
import logging
import concurrent.futures
import threading
from time import sleep
from datetime import datetime

In [ ]:
def get_soup(input_url):
    headers = {
        'User-Agent': 'My User Agent 2.0',
        "Accept-Language": "en-US,en;q=0.5"
    }
    response = requests.get(input_url, headers=headers)
    if response.status_code != 200:
        print("Error in getting link")
        print("response code is : ", response.status_code)
    soup = bs4.BeautifulSoup(response.text, 'html.parser')
    return soup

In [ ]:
def get_person_sum(soup,id):
    info = ''
    p_name=''
    try:
        p_name=soup.select('.col-md-3 .text-center')[0].text.strip()
        info = soup.select('h5')[0].text.strip()
    except:
        info = None
    return [id,p_name,info]

In [ ]:
import re
import logging
import concurrent.futures
import threading
import time
import pandas as pd

lock = threading.Lock()
max_threads = 30

def get_person_info(link, data_list):
    try:
        person_id = re.search(r'profile/(\d+)-', link).group(1)
        site_soup = get_soup('https://www.iranketab.ir' + link)
        with lock:
            data = [id, p_name, info] = get_person_sum(site_soup, person_id)
            data_list.append(data)
    except Exception:
        logging.exception("An error occurred in" + str(link))
        time.sleep(1)

def process_person_info(page_csv, data_list, get_person_info_func):
    df = pd.read_csv(page_csv)
    url_list = df['link'].to_list()
    data_list = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_threads) as executor:
        executor.map(get_person_info_func, url_list, [data_list] * len(url_list))
    return pd.DataFrame(data_list, columns=['id', 'name', 'info'])

translator_info_list = []
translator_info = process_person_info('./translator_page.csv', translator_info_list, get_person_info)
translator_info.to_csv('translator_info.csv', index=False, encoding='utf-8')

writer_info_list = []
writer_info = process_person_info('./writer_page.csv', writer_info_list, get_person_info)
writer_info.to_csv('writer_info.csv', index=False, encoding='utf-8')